# Lab 03 - Spark

## Information

| Student name       | ID       |
|--------------------|----------|
| Nguyễn Thiên Phúc  | 20127681 |
| Lê Nguyễn Nhật Phú | 20127275 |
| Võ Hiền Hải Thuận  | 20127344 |


## Note

**`Restart Kernel` and `Run all cells` before submitting on Moodle or your work will be counted as 0!**


## Assignment

### Import libraries

In [1]:
import findspark 
findspark.init()
findspark.find()
import pyspark
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import RandomForestRegressor

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import scipy
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("").getOrCreate()
from pyspark.python.pyspark.shell import spark

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.4.0
      /_/

Using Python version 3.10.4 (tags/v3.10.4:9d38120, Mar 23 2022 23:13:41)
Spark context Web UI available at http://Phuc-Nguyen:4040
Spark context available as 'sc' (master = local[*], app id = local-1682409599080).
SparkSession available as 'spark'.


### Requirement 1

**Problem 1**: Decision Tree

**Load dataset**

In [2]:
data = spark.read.load("Absenteeism_at_work.csv", format="csv", header=True, delimiter=",")
data = data.withColumn("MOA", data["Month of absence"] - 0).withColumn("label", data['Height'] - 0). \
    withColumn("ROA", data["Reason for absence"] - 0). \
    withColumn("distance", data["Distance from Residence to Work"] - 0). \
    withColumn("BMI", data["Body mass index"] - 0)
data.show(5)

+---+------------------+----------------+---------------+-------+----------------------+-------------------------------+------------+---+----------------------+----------+--------------------+---------+---+--------------+-------------+---+------+------+---------------+-------------------------+---+-----+----+--------+----+
| ID|Reason for absence|Month of absence|Day of the week|Seasons|Transportation expense|Distance from Residence to Work|Service time|Age|Work load Average/day |Hit target|Disciplinary_failure|Education|Son|Social drinker|Social smoker|Pet|Weight|Height|Body mass index|Absenteeism_time_in_hours|MOA|label| ROA|distance| BMI|
+---+------------------+----------------+---------------+-------+----------------------+-------------------------------+------------+---+----------------------+----------+--------------------+---------+---+--------------+-------------+---+------+------+---------------+-------------------------+---+-----+----+--------+----+
| 11|                26| 

**Preprocessing data**

In [3]:
assem = VectorAssembler(inputCols=["MOA", "distance", "ROA", "BMI"], outputCol='features')
data = assem.transform(data)
    
# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)
# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

**Train model**

In [4]:
# Train a DecisionTree model.
dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "indexedLabel", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")

accuracy = evaluator.evaluate(predictions)

y_true = predictions.select("indexedLabel").rdd.flatMap(lambda x: x).collect()
y_pred = predictions.select("prediction").rdd.flatMap(lambda x: x).collect()

+----------+------------+--------------------+
|prediction|indexedLabel|            features|
+----------+------------+--------------------+
|       1.0|         1.0|[6.0,11.0,13.0,29.0]|
|       1.0|         1.0|[12.0,11.0,14.0,2...|
|       1.0|         1.0|[11.0,11.0,18.0,2...|
|       1.0|         1.0|[8.0,11.0,19.0,29.0]|
|       1.0|         1.0|[12.0,11.0,23.0,2...|
+----------+------------+--------------------+
only showing top 5 rows



**Evaluate model**

In [5]:
confusionmatrix = confusion_matrix(y_true, y_pred)

precision = precision_score(y_true, y_pred, average='micro')

recall = recall_score(y_true, y_pred, average='micro')

treeModel = model.stages[2]
# summary only
print(treeModel)
print("Decision Tree - Test Accuracy = %g" % (accuracy))
print("Decision Tree - Test Error = %g" % (1.0 - accuracy))

print("The Confusion Matrix for Decision Tree Model is :\n" + str(confusionmatrix))

print("The precision score for Decision Tree Model is: " + str(precision))

print("The recall score for Decision Tree Model is: " + str(recall))

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_4dee43ed13f5, depth=5, numNodes=27, numClasses=14, numFeatures=4
Decision Tree - Test Accuracy = 0.757426
Decision Tree - Test Error = 0.242574
The Confusion Matrix for Decision Tree Model is :
[[35  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 42  0  0  0  0  1  2  0  1  0  0  0]
 [ 0  0 23  0  0  1  0  6  0  0  0  0  0]
 [ 0  0 14 14  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 13  0  0  3  0  0  0  0  0]
 [ 0  0  0  0  0  9  0  4  0  0  0  0  0]
 [ 0  0  2  0  0  0  3  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  7  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  7  0  0  0  0]
 [ 0  2  0  0  0  0  0  5  0  0  0  0  0]
 [ 0  0  2  0  0  0  0  1  0  0  0  0  0]
 [ 0  0  1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  4  0  0  0  0  0  0]]
The precision score for Decision Tree Model is: 0.7574257425742574
The recall score for Decision Tree Model is: 0.7574257425742574


**Problem 2**: Naive Bayes

**Load dataset**

In [6]:
data = spark.read.load("Absenteeism_at_work.csv", format="csv", header=True, delimiter=",")
data = data.withColumn("MOA", data["Month of absence"] - 0).withColumn("label", data['Seasons'] - 0). \
    withColumn("ROA", data["Reason for absence"] - 0). \
    withColumn("distance", data["Distance from Residence to Work"] - 0). \
    withColumn("BMI", data["Body mass index"] - 0)

**Preprocessing data**

In [7]:
assem = VectorAssembler(inputCols=["label", "MOA"], outputCol='features')

data = assem.transform(data)
# Split the data into train and test
splits = data.randomSplit([0.7, 0.3], 1000)
train = splits[0]
test = splits[1]

**Train model**

In [8]:
# create the trainer and set its parameters
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")

# train the model
model = nb.fit(train)

# select example rows to display.
predictions = model.transform(test)

# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")

y_true = predictions.select("label").rdd.flatMap(lambda x: x).collect()
y_pred = predictions.select("prediction").rdd.flatMap(lambda x: x).collect()
predictions.show(5)

+---+------------------+----------------+---------------+-------+----------------------+-------------------------------+------------+---+----------------------+----------+--------------------+---------+---+--------------+-------------+---+------+------+---------------+-------------------------+----+-----+----+--------+----+----------+--------------------+--------------------+----------+
| ID|Reason for absence|Month of absence|Day of the week|Seasons|Transportation expense|Distance from Residence to Work|Service time|Age|Work load Average/day |Hit target|Disciplinary_failure|Education|Son|Social drinker|Social smoker|Pet|Weight|Height|Body mass index|Absenteeism_time_in_hours| MOA|label| ROA|distance| BMI|  features|       rawPrediction|         probability|prediction|
+---+------------------+----------------+---------------+-------+----------------------+-------------------------------+------------+---+----------------------+----------+--------------------+---------+---+--------------

**Evaluate model**

In [9]:
accuracy = evaluator.evaluate(predictions)

confusionmatrix = confusion_matrix(y_true, y_pred)

precision = precision_score(y_true, y_pred, average='micro')

recall = recall_score(y_true, y_pred, average='micro')


print("Naive Bayes - Test set accuracy = " + str(accuracy))

print("The Confusion Matrix for Naive Bayes Model is :\n" + str(confusionmatrix))

print("The precision score for Naive Bayes Model is: " + str(precision))

print("The recall score for Naive Bayes Model is: " + str(recall))

Naive Bayes - Test set accuracy = 0.061224489795918366
The Confusion Matrix for Naive Bayes Model is :
[[ 0  0  0  0  0]
 [50  0  0  0  0]
 [ 2 33 12  0  0]
 [ 0 22 18  0  0]
 [ 0  0  0 59  0]]
The precision score for Naive Bayes Model is: 0.061224489795918366
The recall score for Naive Bayes Model is: 0.061224489795918366


**Problem 3**: Random Forest

**Load dataset**

In [10]:
data = spark.read.load("Absenteeism_at_work.csv", format="csv", header=True, delimiter=",")

data = data.withColumn("MOA", data["Month of absence"] - 0).withColumn("label", data['Transportation expense'] - 0). \
    withColumn("ROA", data["Reason for absence"] - 0). \
    withColumn("distance", data["Distance from Residence to Work"] - 0). \
    withColumn("BMI", data["Body mass index"] - 0)
data.show(5)

+---+------------------+----------------+---------------+-------+----------------------+-------------------------------+------------+---+----------------------+----------+--------------------+---------+---+--------------+-------------+---+------+------+---------------+-------------------------+---+-----+----+--------+----+
| ID|Reason for absence|Month of absence|Day of the week|Seasons|Transportation expense|Distance from Residence to Work|Service time|Age|Work load Average/day |Hit target|Disciplinary_failure|Education|Son|Social drinker|Social smoker|Pet|Weight|Height|Body mass index|Absenteeism_time_in_hours|MOA|label| ROA|distance| BMI|
+---+------------------+----------------+---------------+-------+----------------------+-------------------------------+------------+---+----------------------+----------+--------------------+---------+---+--------------+-------------+---+------+------+---------------+-------------------------+---+-----+----+--------+----+
| 11|                26| 

**Preprocessing data**

In [11]:
vassembler = VectorAssembler(inputCols=["MOA", "ROA","distance", "BMI"], outputCol='features')
data = vassembler.transform(data)

# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.

labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)

# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

**Train model**

In [12]:
# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=10)

# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

# Chain indexers and forest in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)
#get y_true and y_pred
y_true = predictions.select("indexedLabel").rdd.flatMap(lambda x: x).collect()
y_pred = predictions.select("prediction").rdd.flatMap(lambda x: x).collect()

# Select example rows to display.
predictions.show(5)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")

+---+------------------+----------------+---------------+-------+----------------------+-------------------------------+------------+---+----------------------+----------+--------------------+---------+---+--------------+-------------+---+------+------+---------------+-------------------------+---+-----+----+--------+----+--------------------+------------+--------------------+--------------------+--------------------+----------+--------------+
| ID|Reason for absence|Month of absence|Day of the week|Seasons|Transportation expense|Distance from Residence to Work|Service time|Age|Work load Average/day |Hit target|Disciplinary_failure|Education|Son|Social drinker|Social smoker|Pet|Weight|Height|Body mass index|Absenteeism_time_in_hours|MOA|label| ROA|distance| BMI|            features|indexedLabel|     indexedFeatures|       rawPrediction|         probability|prediction|predictedLabel|
+---+------------------+----------------+---------------+-------+----------------------+----------------

**Evaluate model**

In [13]:
accuracy = evaluator.evaluate(predictions)

confusionmatrix = confusion_matrix(y_true, y_pred)

precision = precision_score(y_true, y_pred, average='micro')

recall = recall_score(y_true, y_pred, average='micro')

rfModel = model.stages[2]
print(rfModel)  # summary only
print("Random Forest - Test Accuracy = %g" % (accuracy))
print("Random Forest - Test Error = %g" % (1.0 - accuracy))

print("The Confusion Matrix for Random Forest Model is :\n" + str(confusionmatrix))

print("The precision score for Random Forest Model is: " + str(precision))

print("The recall score for Random Forest Model is: " + str(recall))


RandomForestClassificationModel: uid=RandomForestClassifier_9fbc943db0d4, numTrees=10, numClasses=24, numFeatures=4
Random Forest - Test Accuracy = 0.862832
Random Forest - Test Error = 0.137168
The Confusion Matrix for Random Forest Model is :
[[60  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 29  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0 22  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  3 14  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 13  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0 11  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  3  9  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  6  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  6  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 10  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 10  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  7  0  0  0  0  0  0 

### Requirement 2

**Dataset**:

**Dataset 1: Iris Dataset**

Description: It includes three iris species with 50 samples each as well as some properties about each flower. One flower species is linearly separable from the other two, but the other two are not linearly separable from each other.

Link  [Iris dataset](https://www.kaggle.com/datasets/uciml/iris)


Information of each column:

| Name          | Meaning         |
|---------------|-----------------|
| Id            | Id of sample    |
| SepalLengthCm | Length of Sepal |
| SepalWidthCm  | Width of Sepal  |
| PetalLengthCm | Length of Petal |
| PetalWidthCm  | Width of Petal  |
| Species       | Specy of sample |

**Dataset 2: Red Wine Dataset**

Description: This datasets is related to red variants of the Portuguese "Vinho Verde" wine. For more details, consult the reference [Cortez et al., 2009]. Due to privacy and logistic issues, only physicochemical (inputs) and sensory (the output) variables are available (e.g. there is no data about grape types, wine brand, wine selling price, etc.).

Link  [Red wine dataset](https://www.kaggle.com/datasets/uciml/red-wine-quality-cortez-et-al-2009)

Information of each column:

| Name                 | Meaning                       |
|----------------------|-------------------------------|
| fixed acidity        | tartaric acid - g / dm^3      |
| volatile acidity     | acetic acid - g / dm^3        |
| citric acid          | g / dm^3                      |
| residual sugar       | g / dm^3                      |
| chlorides            | sodium chloride - g / dm^3    |
| free sulfur dioxide  | mg / dm^3                     |
| total sulfur dioxide | mg / dm^3                     |
| density              | g / cm^3                      |
| pH                   | pH of alcohol                 |
| sulphates            | potassium sulphate - g / dm^3 |
| alcohol              | % by volume                   |
| quality              | score between 0 and 10        |


**Implementation**

**On Iris Dataset**

a. Random forest on Iris dataset

In [15]:
# your code goes here
data = spark.read.load("Iris.csv", format="csv", header=True, delimiter=",")

data = data.withColumn("SepalLength", data["SepalLengthCm"] - 0).withColumn("SepalWidth", data['SepalWidthCm'] - 0). \
    withColumn("PetalLength", data["PetalLengthCm"] - 0). \
    withColumn("PetalWidth", data["PetalWidthCm"] - 0)
data.show(5)

+---+-------------+------------+-------------+------------+-----------+-----------+----------+-----------+----------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|SepalLength|SepalWidth|PetalLength|PetalWidth|
+---+-------------+------------+-------------+------------+-----------+-----------+----------+-----------+----------+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|        5.1|       3.5|        1.4|       0.2|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|        4.9|       3.0|        1.4|       0.2|
|  3|          4.7|         3.2|          1.3|         0.2|Iris-setosa|        4.7|       3.2|        1.3|       0.2|
|  4|          4.6|         3.1|          1.5|         0.2|Iris-setosa|        4.6|       3.1|        1.5|       0.2|
|  5|          5.0|         3.6|          1.4|         0.2|Iris-setosa|        5.0|       3.6|        1.4|       0.2|
+---+-------------+------------+-------------+----------

In [16]:
assem = VectorAssembler(inputCols=["SepalLength", "SepalWidth", "PetalLength", "PetalWidth"], outputCol='features')
data = assem.transform(data)
data.show(5)

+---+-------------+------------+-------------+------------+-----------+-----------+----------+-----------+----------+-----------------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|SepalLength|SepalWidth|PetalLength|PetalWidth|         features|
+---+-------------+------------+-------------+------------+-----------+-----------+----------+-----------+----------+-----------------+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|        5.1|       3.5|        1.4|       0.2|[5.1,3.5,1.4,0.2]|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|        4.9|       3.0|        1.4|       0.2|[4.9,3.0,1.4,0.2]|
|  3|          4.7|         3.2|          1.3|         0.2|Iris-setosa|        4.7|       3.2|        1.3|       0.2|[4.7,3.2,1.3,0.2]|
|  4|          4.6|         3.1|          1.5|         0.2|Iris-setosa|        4.6|       3.1|        1.5|       0.2|[4.6,3.1,1.5,0.2]|
|  5|          5.0|         3.6|          1.4|  

In [17]:
#Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="Species", outputCol="indexedLabel").fit(data)

# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])


In [18]:
# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=10)

# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

In [19]:
# Chain indexers and forest in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)
predictions.show(5)

+---+-------------+------------+-------------+------------+--------------+-----------+----------+-----------+----------+-----------------+------------+-----------------+-----------------+-------------------+----------+---------------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|       Species|SepalLength|SepalWidth|PetalLength|PetalWidth|         features|indexedLabel|  indexedFeatures|    rawPrediction|        probability|prediction| predictedLabel|
+---+-------------+------------+-------------+------------+--------------+-----------+----------+-----------+----------+-----------------+------------+-----------------+-----------------+-------------------+----------+---------------+
|102|          5.8|         2.7|          5.1|         1.9|Iris-virginica|        5.8|       2.7|        5.1|       1.9|[5.8,2.7,5.1,1.9]|         2.0|[5.8,2.7,5.1,1.9]|    [0.0,1.0,9.0]|      [0.0,0.1,0.9]|       2.0| Iris-virginica|
|103|          7.1|         3.0|          5.9|         2.1|I

In [20]:
#get y_true and y_pred
y_true = predictions.select("indexedLabel").rdd.flatMap(lambda x: x).collect()
y_pred = predictions.select("prediction").rdd.flatMap(lambda x: x).collect()

In [21]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")

accuracy = evaluator.evaluate(predictions)

confusionmatrix = confusion_matrix(y_true, y_pred)

precision = precision_score(y_true, y_pred, average='micro')

recall = recall_score(y_true, y_pred, average='micro')

rfModel = model.stages[2]
print(rfModel)  # summary only
print("Random Forest - Test Accuracy = %g" % (accuracy))
print("Random Forest - Test Error = %g" % (1.0 - accuracy))

print("The Confusion Matrix for Random Forest Model is :\n" + str(confusionmatrix))

print("The precision score for Random Forest Model is: " + str(precision))

print("The recall score for Random Forest Model is: " + str(recall))

RandomForestClassificationModel: uid=RandomForestClassifier_7dbf9b25b2e6, numTrees=10, numClasses=3, numFeatures=4
Random Forest - Test Accuracy = 0.9375
Random Forest - Test Error = 0.0625
The Confusion Matrix for Random Forest Model is :
[[12  0  0]
 [ 0 16  1]
 [ 0  2 17]]
The precision score for Random Forest Model is: 0.9375
The recall score for Random Forest Model is: 0.9375


b. Decision Tree on Iris dataset

In [22]:
#Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="Species", outputCol="indexedLabel").fit(data)

# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

In [23]:
# Train a DecisionTree model.
dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")

accuracy = evaluator.evaluate(predictions)

In [24]:
#get y_true and y_pred
y_true = predictions.select("indexedLabel").rdd.flatMap(lambda x: x).collect()
y_pred = predictions.select("prediction").rdd.flatMap(lambda x: x).collect()

In [25]:
confusionmatrix = confusion_matrix(y_true, y_pred)

precision = precision_score(y_true, y_pred, average='micro')

recall = recall_score(y_true, y_pred, average='micro')

treeModel = model.stages[2]
# summary only
print(treeModel)
print("Decision Tree - Test Accuracy = %g" % (accuracy))
print("Decision Tree - Test Error = %g" % (1.0 - accuracy))

print("The Confusion Matrix for Decision Tree Model is :\n" + str(confusionmatrix))

print("The precision score for Decision Tree Model is: " + str(precision))

print("The recall score for Decision Tree Model is: " + str(recall))

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_688db1972223, depth=5, numNodes=11, numClasses=3, numFeatures=4
Decision Tree - Test Accuracy = 0.930233
Decision Tree - Test Error = 0.0697674
The Confusion Matrix for Decision Tree Model is :
[[14  0  0]
 [ 0 10  2]
 [ 0  1 16]]
The precision score for Decision Tree Model is: 0.9302325581395349
The recall score for Decision Tree Model is: 0.9302325581395349


**On Red Wine Dataset**

a. Linear Regression on Iris dataset

In [41]:
data = spark.read.load("winequality-red.csv", format="csv", header=True, delimiter=";")
data = data.withColumn("fixed acidity", data["fixed acidity"] - 0).withColumn("volatile acidity", data['volatile acidity'] - 0). \
    withColumn("citric acid", data["citric acid"] - 0). \
    withColumn("residual sugar", data["residual sugar"] - 0). \
    withColumn("chlorides", data["chlorides"] - 0). \
    withColumn("free sulfur dioxide", data["free sulfur dioxide"] - 0). \
    withColumn("total sulfur dioxide", data["total sulfur dioxide"] - 0). \
    withColumn("density", data["density"] - 0). \
    withColumn("pH", data["pH"] - 0). \
    withColumn("sulphates", data["sulphates"] - 0). \
    withColumn("alcohol", data["alcohol"] - 0). \
    withColumn("quality", data["quality"] - 0)
data.show(5)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|quality|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|          7.4|             0.7|        0.0|           1.9|    0.076|               11.0|                34.0| 0.9978|3.51|     0.56|    9.4|    5.0|
|          7.8|            0.88|        0.0|           2.6|    0.098|               25.0|                67.0| 0.9968| 3.2|     0.68|    9.8|    5.0|
|          7.8|            0.76|       0.04|           2.3|    0.092|               15.0|                54.0|  0.997|3.26|     0.65|    9.8|    5.0|
|         11.2|            0.28|       0.56|           1.9|    0.075|               17.0|           

In [42]:
assem = VectorAssembler(inputCols=["fixed acidity", "volatile acidity", "citric acid", "residual sugar",
                                    "chlorides", "free sulfur dioxide","total sulfur dioxide", "density", 
                                    "pH", "sulphates", "alcohol"], outputCol='features')
data = assem.transform(data)
data.printSchema()

root
 |-- fixed acidity: double (nullable = true)
 |-- volatile acidity: double (nullable = true)
 |-- citric acid: double (nullable = true)
 |-- residual sugar: double (nullable = true)
 |-- chlorides: double (nullable = true)
 |-- free sulfur dioxide: double (nullable = true)
 |-- total sulfur dioxide: double (nullable = true)
 |-- density: double (nullable = true)
 |-- pH: double (nullable = true)
 |-- sulphates: double (nullable = true)
 |-- alcohol: double (nullable = true)
 |-- quality: double (nullable = true)
 |-- features: vector (nullable = true)



In [43]:
#Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="quality", outputCol="indexedLabel").fit(data)

# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=12).fit(data)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

In [44]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(labelCol="quality", featuresCol="features")

# Fit the model
lrModel = lr.fit(trainingData)

In [45]:
test_stats = lrModel.evaluate(testData)
print(f"RMSE: {test_stats.rootMeanSquaredError}")
print(f"R2: {test_stats.r2}")
print(f"MSE: {test_stats.meanSquaredError}")

RMSE: 0.6450940417336557
R2: 0.3247853702061759
MSE: 0.4161463226802636


b. Decision Tree regression on wine dataset

In [46]:
data = spark.read.format("csv").load("winequality-red.csv", header=True, delimiter=";")

data = data.withColumn("FA", data["fixed acidity"] - 0).withColumn("label", data['quality'] - 0). \
    withColumn("CA", data["citric acid"] - 0). \
    withColumn("RS", data["residual sugar"] - 0). \
    withColumn("A", data["alcohol"] - 0)
data.show(5)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+----+-----+----+---+---+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|quality|  FA|label|  CA| RS|  A|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+----+-----+----+---+---+
|          7.4|             0.7|          0|           1.9|    0.076|                 11|                  34| 0.9978|3.51|     0.56|    9.4|      5| 7.4|  5.0| 0.0|1.9|9.4|
|          7.8|            0.88|          0|           2.6|    0.098|                 25|                  67| 0.9968| 3.2|     0.68|    9.8|      5| 7.8|  5.0| 0.0|2.6|9.8|
|          7.8|            0.76|       0.04|           2.3|    0.092|                 15|                  54|  0.997|3.26|     0.

In [47]:
vassembler = VectorAssembler(inputCols=["FA", "CA", "RS","A"], outputCol='features')

data = vassembler.transform(data)
data.show(5)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+----+-----+----+---+---+-------------------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|quality|  FA|label|  CA| RS|  A|           features|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+----+-----+----+---+---+-------------------+
|          7.4|             0.7|          0|           1.9|    0.076|                 11|                  34| 0.9978|3.51|     0.56|    9.4|      5| 7.4|  5.0| 0.0|1.9|9.4|  [7.4,0.0,1.9,9.4]|
|          7.8|            0.88|          0|           2.6|    0.098|                 25|                  67| 0.9968| 3.2|     0.68|    9.8|      5| 7.8|  5.0| 0.0|2.6|9.8|  [7.8,0.0,2.6,9.8]|
|          7.8|            0.7

In [48]:
# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

In [49]:
# Train a DecisionTree model.
dt = DecisionTreeRegressor(featuresCol="indexedFeatures")

# Chain indexer and tree in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, dt])

# Train model.  This also runs the indexer.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)
predictions.show(5)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+----+-----+----+---+----+--------------------+--------------------+-----------------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|quality|  FA|label|  CA| RS|   A|            features|     indexedFeatures|       prediction|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+----+-----+----+---+----+--------------------+--------------------+-----------------+
|           10|            0.32|       0.59|           2.2|    0.077|                  3|                  15| 0.9994| 3.2|     0.78|    9.6|      5|10.0|  5.0|0.59|2.2| 9.6| [10.0,0.59,2.2,9.6]| [10.0,0.59,2.2,9.6]|5.048780487804878|
|           10|            0.41|       0.45|           6.2| 

In [50]:
# Select (prediction, true label) and compute test error
evaluator_rmse = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator_rmse.evaluate(predictions)

evaluator_r2 = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName= "r2")
r2 = evaluator_r2.evaluate(predictions)

print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
print("Coefficient of Determination (R^2) on test data = %g" % r2)

treeModel = model.stages[1]
# summary only
print(treeModel)

Root Mean Squared Error (RMSE) on test data = 0.731559
Coefficient of Determination (R^2) on test data = 0.179567
DecisionTreeRegressionModel: uid=DecisionTreeRegressor_8e4293706f2c, depth=5, numNodes=63, numFeatures=4


c. Random Forest regression on wine dataset

In [51]:
data = spark.read.format("csv").load("winequality-red.csv", header=True, delimiter=";")

data = data.withColumn("FA", data["fixed acidity"] - 0).withColumn("label", data['quality'] - 0). \
    withColumn("CA", data["citric acid"] - 0). \
    withColumn("RS", data["residual sugar"] - 0). \
    withColumn("A", data["alcohol"] - 0)

In [52]:
vassembler = VectorAssembler(inputCols=["FA", "CA", "RS","A"], outputCol='features')

data = vassembler.transform(data)
data.show(5)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+----+-----+----+---+---+-------------------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|quality|  FA|label|  CA| RS|  A|           features|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+----+-----+----+---+---+-------------------+
|          7.4|             0.7|          0|           1.9|    0.076|                 11|                  34| 0.9978|3.51|     0.56|    9.4|      5| 7.4|  5.0| 0.0|1.9|9.4|  [7.4,0.0,1.9,9.4]|
|          7.8|            0.88|          0|           2.6|    0.098|                 25|                  67| 0.9968| 3.2|     0.68|    9.8|      5| 7.8|  5.0| 0.0|2.6|9.8|  [7.8,0.0,2.6,9.8]|
|          7.8|            0.7

In [53]:
# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

In [54]:
# Train a RandomForest model.
rf = RandomForestRegressor(featuresCol="indexedFeatures")

# Chain indexer and forest in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, rf])

# Train model.  This also runs the indexer.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.show(5)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+----+-----+----+---+----+--------------------+--------------------+-----------------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|quality|  FA|label|  CA| RS|   A|            features|     indexedFeatures|       prediction|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+----+-----+----+---+----+--------------------+--------------------+-----------------+
|           10|            0.29|        0.4|           2.9|    0.098|                 10|                  26| 1.0006|3.48|     0.91|    9.7|      5|10.0|  5.0| 0.4|2.9| 9.7|  [10.0,0.4,2.9,9.7]|  [10.0,0.4,2.9,9.7]|5.302852403799031|
|           10|            0.31|       0.47|           2.6| 

In [55]:
# Select (prediction, true label) and compute test error
evaluator_rmse = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator_rmse.evaluate(predictions)

evaluator_r2 = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName= "r2")
r2 = evaluator_r2.evaluate(predictions)

print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
print("Coefficient of Determination (R^2) on test data = %g" % r2)

rfModel = model.stages[1]
print(rfModel)  # summary only

Root Mean Squared Error (RMSE) on test data = 0.649571
Coefficient of Determination (R^2) on test data = 0.287167
RandomForestRegressionModel: uid=RandomForestRegressor_5b0f3a01dae8, numTrees=20, numFeatures=4
